Notebook dedicado a explorar métodos de lectura y escritura de un Dataset. Para ello se trabajara con Los datos obtenidos del fichero del portal de datos abiertos del ayuntamiento de Madrid, especificamente el Padron de Madrid, los cuales han están almacenados en una tabla en formato Parquet.

En este Notebook leeremos datos en distintos formatos:
* Parquet

Escribiremos datos en los siguientes formatos:
* CSV
* JSON

In [1]:
case class Padron (
  COD_DISTRITO: Integer,
  DESC_DISTRITO: String,
  COD_DIST_BARRIO: Integer,
  DESC_BARRIO: String,
  COD_BARRIO: Integer,
  COD_DIST_SECCION: Integer,
  COD_SECCION: Integer,
  COD_EDAD_INT: Integer,
  EspanolesHombres: Integer,
  EspanolesMujeres: Integer,
  ExtranjerosHombres: Integer,
  ExtranjerosMujeres: Integer
  )

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.1.35:4041
SparkContext available as 'sc' (version = 3.0.2, master = local[*], app id = local-1614555093351)
SparkSession available as 'spark'


defined class Padron


In [4]:
import spark.implicits._

// Leemos Datos procedentes de tabla cuyos datos están en formato parquet
// una vez leidos transformamos el DataFrame en un Dataset
// val sourceParquetDS = spark.table("default.padron_madrid").as[Padron].cache()
val sourceParquetDS = spark.read.parquet("./dataOutput/df/parquet").as[Padron].cache()

// obtenemos el total de registros
val totalRecords = sourceParquetDS.count()

import spark.implicits._
sourceParquetDS: org.apache.spark.sql.Dataset[Padron] = [COD_DISTRITO: int, DESC_DISTRITO: string ... 10 more fields]
totalRecords: Long = 237675


In [5]:
import org.apache.spark.sql.functions._

val totalBySexAndTypeDF = sourceParquetDS.withColumn("Hombres", col("EspanolesHombres") + col("ExtranjerosHombres"))
                                       .withColumn("Mujeres", col("EspanolesMujeres") + col("ExtranjerosMujeres"))

val distritoAndBarrioPeopleAggDF = totalBySexAndTypeDF.groupBy("DESC_DISTRITO", "DESC_BARRIO")
.agg(
  sum("EspanolesHombres").alias("TotalEspanolesHombres"),
  sum("EspanolesMujeres").alias("TotalEspanolesMujeres"),
  sum("ExtranjerosHombres").alias("TotalExtranjerosHombres"),
  sum("ExtranjerosMujeres").alias("TotalExtranjerosMujeres"),
  sum("Hombres").alias("TotalHombres"),
  sum("Mujeres").alias("TotalMujeres")
  )
.withColumn("TotalPersonas", col("TotalHombres") + col("TotalMujeres"))
.select("DESC_DISTRITO", "DESC_BARRIO", "TotalEspanolesHombres", "TotalEspanolesMujeres", "TotalExtranjerosHombres", "TotalExtranjerosMujeres", "TotalHombres", "TotalMujeres", "TotalPersonas").cache()

distritoAndBarrioPeopleAggDF.orderBy(col("TotalPersonas").desc).show()

+-------------------+--------------------+---------------------+---------------------+-----------------------+-----------------------+------------+------------+-------------+
|      DESC_DISTRITO|         DESC_BARRIO|TotalEspanolesHombres|TotalEspanolesMujeres|TotalExtranjerosHombres|TotalExtranjerosMujeres|TotalHombres|TotalMujeres|TotalPersonas|
+-------------------+--------------------+---------------------+---------------------+-----------------------+-----------------------+------------+------------+-------------+
|             LATINA|              ALUCHE|                25257|                29932|                   5592|                   6609|       30849|       36541|        67390|
|FUENCARRAL-EL PARDO|            VALVERDE|                26976|                29226|                   3730|                   4515|       30706|       33741|        64447|
|          HORTALEZA|        VALDEFUENTES|                28206|                28615|                   2824|               

import org.apache.spark.sql.functions._
totalBySexAndTypeDF: org.apache.spark.sql.DataFrame = [COD_DISTRITO: int, DESC_DISTRITO: string ... 12 more fields]
distritoAndBarrioPeopleAggDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [DESC_DISTRITO: string, DESC_BARRIO: string ... 7 more fields]


In [6]:
val distritoPeopleAggDF = distritoAndBarrioPeopleAggDF.groupBy("DESC_DISTRITO").agg(
sum("TotalEspanolesHombres").alias("TotalEspanolesHombres"),
  sum("TotalEspanolesMujeres").alias("TotalEspanolesMujeres"),
  sum("TotalExtranjerosHombres").alias("TotalExtranjerosHombres"),
  sum("TotalExtranjerosMujeres").alias("TotalExtranjerosMujeres"),
  sum("TotalHombres").alias("TotalHombres"),
  sum("TotalMujeres").alias("TotalMujeres"),
  sum("TotalPersonas").alias("TotalPersonas")
)
.withColumn("TotalEspanoles", col("TotalEspanolesHombres") + col("TotalEspanolesMujeres"))
.withColumn("TotalExtranjeros", col("TotalExtranjerosHombres") + col("TotalExtranjerosMujeres"))

distritoPeopleAggDF.orderBy(col("TotalPersonas").desc).show()

+-------------------+---------------------+---------------------+-----------------------+-----------------------+------------+------------+-------------+--------------+----------------+
|      DESC_DISTRITO|TotalEspanolesHombres|TotalEspanolesMujeres|TotalExtranjerosHombres|TotalExtranjerosMujeres|TotalHombres|TotalMujeres|TotalPersonas|TotalEspanoles|TotalExtranjeros|
+-------------------+---------------------+---------------------+-----------------------+-----------------------+------------+------------+-------------+--------------+----------------+
|        CARABANCHEL|                94274|               108948|                  27018|                  29908|      121292|      138856|       260148|        203222|           56926|
|FUENCARRAL-EL PARDO|               107136|               118780|                  10034|                  13021|      117170|      131801|       248971|        225916|           23055|
|             LATINA|                92207|               106512|     

distritoPeopleAggDF: org.apache.spark.sql.DataFrame = [DESC_DISTRITO: string, TotalEspanolesHombres: bigint ... 8 more fields]


In [7]:
import org.apache.spark.sql.types.DecimalType

val distritoPeopleAggRatioDF = distritoPeopleAggDF
.withColumn("%_Espanoles", (lit(100) * col("TotalEspanoles") / col("TotalPersonas")).cast(DecimalType(4,2)))
.withColumn("%_Extranjeros", (lit(100) * col("TotalExtranjeros") / col("TotalPersonas")).cast(DecimalType(4,2)))
.withColumn("%_Mujeres", (lit(100) * col("TotalMujeres") / col("TotalPersonas")).cast(DecimalType(4,2)))
.withColumn("%_Hombres", (lit(100) * col("TotalHombres") / col("TotalPersonas")).cast(DecimalType(4,2)))

distritoPeopleAggRatioDF.show()

+-------------------+---------------------+---------------------+-----------------------+-----------------------+------------+------------+-------------+--------------+----------------+-----------+-------------+---------+---------+
|      DESC_DISTRITO|TotalEspanolesHombres|TotalEspanolesMujeres|TotalExtranjerosHombres|TotalExtranjerosMujeres|TotalHombres|TotalMujeres|TotalPersonas|TotalEspanoles|TotalExtranjeros|%_Espanoles|%_Extranjeros|%_Mujeres|%_Hombres|
+-------------------+---------------------+---------------------+-----------------------+-----------------------+------------+------------+-------------+--------------+----------------+-----------+-------------+---------+---------+
|             LATINA|                92207|               106512|                  19780|                  23149|      111987|      129661|       241648|        198719|           42929|      82.23|        17.77|    53.66|    46.34|
|             TETUAN|                58796|                69116|       

import org.apache.spark.sql.types.DecimalType
distritoPeopleAggRatioDF: org.apache.spark.sql.DataFrame = [DESC_DISTRITO: string, TotalEspanolesHombres: bigint ... 12 more fields]


In [8]:
// examinamos la estructura (esquema/schema) del DF final
distritoPeopleAggRatioDF.printSchema

root
 |-- DESC_DISTRITO: string (nullable = true)
 |-- TotalEspanolesHombres: long (nullable = true)
 |-- TotalEspanolesMujeres: long (nullable = true)
 |-- TotalExtranjerosHombres: long (nullable = true)
 |-- TotalExtranjerosMujeres: long (nullable = true)
 |-- TotalHombres: long (nullable = true)
 |-- TotalMujeres: long (nullable = true)
 |-- TotalPersonas: long (nullable = true)
 |-- TotalEspanoles: long (nullable = true)
 |-- TotalExtranjeros: long (nullable = true)
 |-- %_Espanoles: decimal(4,2) (nullable = true)
 |-- %_Extranjeros: decimal(4,2) (nullable = true)
 |-- %_Mujeres: decimal(4,2) (nullable = true)
 |-- %_Hombres: decimal(4,2) (nullable = true)



In [9]:
import java.math.BigDecimal
// import scala.math.BigInt

// Dado el schema del DF anterior
// Los decimal se expresan utilizando como tipo de dato la clase java.math.BigDecimal
// Los datos de tipo long podemos expresarlos con datos del tipo Long y scala.math.BigInt. 
// El primero arroja mejor rendimiento mientras que el segundo permite almacenar un rango más amplio que el tipo Long

case class PadronDistritoAggregation (
  DESC_DISTRITO: String,
  TotalEspanolesHombres: Long,
  TotalEspanolesMujeres: Long,
  TotalExtranjerosHombres: Long,
  TotalExtranjerosMujeres: Long,
  TotalHombres: Long,
  TotalMujeres: Long,
  TotalPersonas: Long,
  TotalEspanoles: Long,
  TotalExtranjeros: Long,
  `%_Espanoles`: BigDecimal,
  `%_Extranjeros`: BigDecimal,
  `%_Mujeres`: BigDecimal,
  `%_Hombres`: BigDecimal
)

import java.math.BigDecimal
defined class PadronDistritoAggregation


In [10]:
//convertimos el DataFrame = Dataset[Row] a Dataset[PadronDistritoAggregation]
val distritoPeopleAggDS = distritoPeopleAggRatioDF.as[PadronDistritoAggregation]
distritoPeopleAggDS.show()

+-------------------+---------------------+---------------------+-----------------------+-----------------------+------------+------------+-------------+--------------+----------------+-----------+-------------+---------+---------+
|      DESC_DISTRITO|TotalEspanolesHombres|TotalEspanolesMujeres|TotalExtranjerosHombres|TotalExtranjerosMujeres|TotalHombres|TotalMujeres|TotalPersonas|TotalEspanoles|TotalExtranjeros|%_Espanoles|%_Extranjeros|%_Mujeres|%_Hombres|
+-------------------+---------------------+---------------------+-----------------------+-----------------------+------------+------------+-------------+--------------+----------------+-----------+-------------+---------+---------+
|             LATINA|                92207|               106512|                  19780|                  23149|      111987|      129661|       241648|        198719|           42929|      82.23|        17.77|    53.66|    46.34|
|             TETUAN|                58796|                69116|       

distritoPeopleAggDS: org.apache.spark.sql.Dataset[PadronDistritoAggregation] = [DESC_DISTRITO: string, TotalEspanolesHombres: bigint ... 12 more fields]


In [11]:
println(s"Número actual de particiones: ${distritoPeopleAggDS.rdd.getNumPartitions}")

// reducimos el numero de partitiones para que de esta forma sea un único fichero JSON o CSV el que sea generado al llevar a cabo la escritura
val onePartitionDS = distritoPeopleAggDS.coalesce(1)

println(s"Número actual de particiones después del coalesce: ${onePartitionDS.rdd.getNumPartitions}")

onePartitionDS.write.mode("overwrite").format("csv").option("header", true).save("./dataOutput/ds/csv")
onePartitionDS.write.mode("overwrite").json("./dataOutput/ds/json")

Número actual de particiones: 200
Número actual de particiones después del coalesce: 1


onePartitionDS: org.apache.spark.sql.Dataset[PadronDistritoAggregation] = [DESC_DISTRITO: string, TotalEspanolesHombres: bigint ... 12 more fields]
